In [5]:
from PIL import Image
import numpy as np
import cv2

In [2]:
im = Image.open('101087.jpg')

In [3]:
im = np.array(im)

# scale im from 0-255 to 0-1
im = im / 255
tx, ty, nchan = im.shape
orig_sz = [tx, ty] 

# default feature weights
weights = [0, 0, 0.0039, 0.0050, 0.0058, 0.0069, 0.0040, 0.0044, 0.0049, 0.0024, 0.0027, 0.0170, 0.0074]

In [27]:
# resize factor in (0, 1], to speed up eigenvector computation
rsz = 1

In [ ]:
def multiscale_Pb(im, rsz):
    """
    compute local contour cues of an image.
    im: 3 channel image arrays with value scaled between (0,1].
    rsz: resize factor.
    """
    [bg1, bg2, bg3, cga1, cga2, cga3, cgb1, cgb2, cgb3, tg1, tg2, tg3, textons] = gradient_mPb(im)
    

In [83]:
def gradients_mPb(im):
    """
    compute image gradients.
    im: 3 channel image arrays with value scaled between (0,1].
    """
    # parameters - bin and smooth
    num_orient = 8
    num_L_bins = 25
    num_a_bins = 25
    num_b_bins = 25
    bg_smooth_sigma = 0.1
    cg_smooth_sigma = 0.05
    border = 30
    sigma_tg_filter_sm = 30
    sigma_tg_filter_lg = np.sqrt(2) * 2
    
    # parameters - radius
    num_bg_radius = 3
    num_cg_radius = 3
    num_tg_radius = 3
    bg_radius = [3, 5, 10]
    cg_radius = [5, 10, 20]
    tg_radius = [5, 10, 20]
    
    # compute bg histogram smoothing kernel
    bg_smooth_kernel = lib_image.gaussian(bg_smooth_sigma*num_L_bins)
    cga_smooth_kernel = lib_image.gaussian(cg_smooth_sigma*num_a_bins)
    cgb_smooth_kernel = lib_image.gaussian(cg_smooth_sigma*num_b_bins)
    
    # get image
    L = im[0]
    a = im[1]
    b = im[2]
    
    # mirror border
    L = lib_image.border_mirror_2D(L, border)
    a = lib_image.border_mirror_2D(a, border)
    b = lib_image.border_mirror_2D(b, border)
    
    return L, a, b
    

In [40]:
np.zeros([3,1])

array([[0.],
       [0.],
       [0.]])

In [87]:
class lib_image(object):

    def gaussian(sigma=1, deriv=0): 
        """
        Gaussian kernel(1D)
        The length of the returned vector is 2*support+1.
        The support defaults to 3*sigma.
        The kernel is normalized to have unit L1 norm.
        If returning a 1st or 2nd derivative, the kernel has zero mean
        """
        support = np.ceil(3*sigma)
        support = support.astype(int)
        # Enlarge support so that hilbert transform can be done efficiently
        support_big = support
        
        # compute constants
        sigma2_inv = 1 / (sigma*sigma)
        
        # compute gaussian (or gaussian derivative)
        size = 2*support_big + 1
        m = np.zeros([size, 1])
        x = -support_big
        if (deriv == 0):
            for n in range(0, size):
                m[n][0] = np.exp(x*x*(-0.5*sigma2_inv))
                x += 1
        elif (deriv == 1):
            for n in range(0, size):
                m[n][0] = np.exp(x*x*(-0.5*sigma2_inv)) * (-x)
                x += 1
        elif (deriv == 2):
            for n in range(0, size):
                m[n][0] = np.exp(x*x*(-0.5*sigma2_inv)) * (x*x*sigma2_inv - 1)
        else:
            raise ValueError("Only derivatives 0, 1, 2 supported")
            
        # make zero mean (if returning derivative)
        if (deriv > 0):
            m -= np.mean(m)
            
        # make unit L1 norm
        m /= np.sum(np.abs(m))
        
        return m
    
    def border_mirror_2D(m, size):
        """
        Expand the border of the 2D matrix by the specified sizes in the X and Y dimensions.
        The expand border is filled with the mirror image of interior data.
        """
        # check that matrix is 2D
        if (len(m.shape) != 2):
            raise ValueError("matrix must be 2D")
            
        # get matrix size
        size_x, size_y = m.shape
        
        # compute border of X and Y
        border_x, border_y = size, size
        
        # check that interior can be mirrored
        if (border_x > size_x or border_y > size_x):
            raise ValueError("cannot create mirrored border larger than matrix interior dimensions")
            
        # mirror border
        m_dst = lib_image.compute_border_mirror_2D(m, border_x, border_y)
        
        return m_dst
    
    def compute_border_mirror_2D(m, border_x, border_y):
        """
        compute border mirrored matrix
        """
        return np.pad(m, [border_x, border_y], 'reflect')

In [88]:
L, a, b = gradients_mPb(im)

In [89]:
L

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
        0.99607843],
       [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
        0.99607843],
       ...,
       [0.0627451 , 0.13333333, 0.1372549 , ..., 0.0627451 , 0.13333333,
        0.1372549 ],
       [0.0745098 , 0.14509804, 0.14901961, ..., 0.0745098 , 0.14509804,
        0.14901961],
       [0.0745098 , 0.14509804, 0.14901961, ..., 0.0745098 , 0.14509804,
        0.14901961]])

In [71]:
a

4

In [1]:
import scipy

In [2]:
scipy.__version__

'1.4.1'

In [22]:
def rotate_2D_crop(m, orient, size_x, size_y):
        """
        rotate matrix by some orientation
        """

        m_rotate = np.zeros([size_x, size_y])
        # center of m
        ox, oy = m.shape[1] // 2, m.shape[0] // 2

        # center of m_rotate
        ox_r, oy_r = size_y // 2, size_x // 2

        # compute the matrix after rotation
        for i in range(m.shape[0]):
            for j in range(m.shape[1]):
                x_coord = j - ox
                y_coord = oy - i
                dist = np.sqrt((x_coord)**2 + (y_coord)**2)
                theta = np.arctan2(x_coord, y_coord) + orient
                new_y = np.ceil(dist * np.cos(theta)).astype(int) + ox_r
                new_x = -np.ceil(dist * np.sin(theta)).astype(int) + oy_r
                m_rotate[new_x][new_y] = m[i][j]
        x_start = (m_rotate.shape[0] - m.shape[0]) // 2
        y_start = (m_rotate.shape[1] - m.shape[1]) // 2
        m_rotate = m_rotate[x_start: x_start+m.shape[0], y_start: y_start+m.shape[1]] 

        return m_rotate
        


In [25]:
m = np.array([[0,1,0],[0,1,0],[1,1,0]])
m

array([[0, 1, 0],
       [0, 1, 0],
       [1, 1, 0]])

In [32]:
rotate_2D_crop(m, 0, 7, 7)

array([[1., 0., 0.],
       [0., 1., 1.],
       [1., 0., 0.]])

In [21]:
np.zeros([7,7])[1:3,1:3]

array([[0., 0.],
       [0., 0.]])

In [41]:
np.round(1.414 * np.cos(-0.785))

1.0

In [39]:
np.round(0.998).astype(int)

1

In [40]:
np.round(0.998)

1.0

In [47]:
np.arctan2(1, 1)

0.7853981633974483